In [1]:
import numpy
from scipy import linalg, special, integrate, optimize
from scipy.integrate import solve_ivp
import matplotlib
from matplotlib import pyplot
import pandas
import gc
import os

from tqdm.notebook import tqdm

import joblib

import bmme

In [8]:
omega = .3
lamda = .03                            # .03, .15, .3, .6 eV
epsilon = .15  
Gamma = .01                            # .01, .026, .05 eV
T = .026

fix_epsilon = True
if fix_epsilon:
    e1 = epsilon
    epsilon += lamda**2 / omega
    
def file_apx():
    return f"{'ef' if fix_epsilon else 'e'}{(e1 if fix_epsilon else epsilon):.3f}_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}"

## Migrate old Langevin data

In [3]:
voltages = numpy.arange(0, 1001, 10)
x1 = numpy.linspace(-100, 100, 2001)

mean_force = numpy.load(f"data/holstein_semicl/mean_force_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy")
friction = numpy.load(f"data/holstein_semicl/friction_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy")
correlation = numpy.load(f"data/holstein_semicl/correlation_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy")
currents = numpy.load(f"data/holstein_semicl/currents_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy")

mean_force = mean_force.flatten()
friction = friction.flatten()
correlation = correlation.flatten()
current_L = currents[:,:,0].flatten()
current_R = currents[:,:,1].flatten()

voltages_new = numpy.array(numpy.repeat(voltages, len(x1)), dtype=numpy.int32)
x1_new = numpy.tile(x1, len(voltages))

panda = pandas.DataFrame(data={"voltage": voltages_new,
                               "X": x1_new,
                               "current_L": current_L,
                               "current_R": current_R,
                               "mean_force": mean_force,
                               "friction": friction,
                               "diffusion": correlation})

print(panda.dtypes)

panda.to_csv(f"data/holstein_semicl/langevin_bmme_{file_apx()}.csv")

os.rename(f"data/holstein_semicl/mean_force_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy", f"data/holstein_semicl/mean_force_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy.migrated")
os.rename(f"data/holstein_semicl/friction_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy", f"data/holstein_semicl/friction_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy.migrated")
os.rename(f"data/holstein_semicl/correlation_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy", f"data/holstein_semicl/diffusion_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy.migrated")
os.rename(f"data/holstein_semicl/currents_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy", f"data/holstein_semicl/currents_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy.migrated")

voltage         int32
X             float64
current_L     float64
current_R     float64
mean_force    float64
friction      float64
diffusion     float64
dtype: object


## Migrate Sam's Langevin data

In [74]:
voltages = numpy.arange(0, 1001, 100)

voltages_new = []
x1_new = []
mean_force = []
friction = []
correlation = []
current_L = []

for voltage in voltages:
    data0 = numpy.genfromtxt(f"data/holstein_semicl/harmonic_omega_{omega:.3f}eV_lambda_{lamda:.2f}eV/gamma_{(1e3*Gamma):.0f}meV_temp_{T:.3f}eV/voltage_{voltage*1e-3:.1f}eV/tier_2/current_heom.txt").T
    grid = data0[0]
    heom_current = data0[1]
    heom_mean_force = numpy.genfromtxt(f"data/holstein_semicl/harmonic_omega_{omega:.3f}eV_lambda_{lamda:.2f}eV/gamma_{(1e3*Gamma):.0f}meV_temp_{T:.3f}eV/voltage_{voltage*1e-3:.1f}eV/tier_2/average_electronic_force_heom.txt").T[1]
    heom_friction = numpy.genfromtxt(f"data/holstein_semicl/harmonic_omega_{omega:.3f}eV_lambda_{lamda:.2f}eV/gamma_{(1e3*Gamma):.0f}meV_temp_{T:.3f}eV/voltage_{voltage*1e-3:.1f}eV/tier_2/friction_heom.txt").T[1]
    heom_diffusion = numpy.genfromtxt(f"data/holstein_semicl/harmonic_omega_{omega:.3f}eV_lambda_{lamda:.2f}eV/gamma_{(1e3*Gamma):.0f}meV_temp_{T:.3f}eV/voltage_{voltage*1e-3:.1f}eV/tier_2/diffusion_heom.txt").T[1]
    grid_ext1 = numpy.arange(grid[0], -100, grid[0] - grid[1])[::-1][:-1]
    grid_ext2 = numpy.arange(grid[-1], 100, grid[-1] - grid[-2])[1:]
    
    voltages_new.extend([voltage] * (len(grid_ext1) + len(grid) + len(grid_ext2)))
    x1_new.extend(grid_ext1); x1_new.extend(grid); x1_new.extend(grid_ext2)
    mean_force.extend(-omega * grid_ext1 - lamda * numpy.sqrt(2))
    mean_force.extend(-omega * grid + heom_mean_force)
    mean_force.extend(-omega * grid_ext2)
    friction.extend(numpy.zeros(len(grid_ext1)))
    friction.extend(heom_friction)
    friction.extend(numpy.zeros(len(grid_ext2)))
    correlation.extend(numpy.zeros(len(grid_ext1)))
    correlation.extend(heom_diffusion)
    correlation.extend(numpy.zeros(len(grid_ext2)))
    current_L.extend(numpy.zeros(len(grid_ext1)))
    current_L.extend(heom_current)
    current_L.extend(numpy.zeros(len(grid_ext2)))
    
voltages_new = numpy.array(numpy.array(voltages_new).flatten(), dtype=numpy.int32)
x1_new = numpy.array(x1_new).flatten()
mean_force = numpy.array(mean_force).flatten()
friction = numpy.array(friction).flatten()
correlation = numpy.array(correlation).flatten()
current_L = numpy.array(current_L).flatten()

panda = pandas.DataFrame(data={"voltage": voltages_new,
                               "X": x1_new,
                               "current_L": current_L,
                               "mean_force": mean_force,
                               "friction": friction,
                               "diffusion": correlation})

print(panda.dtypes)

panda.to_csv(f"data/holstein_semicl/langevin_heom_{file_apx()}.csv")

voltage         int32
X             float64
current_L     float64
mean_force    float64
friction      float64
diffusion     float64
dtype: object


## Migrate old classical averages

In [23]:
voltages = numpy.arange(0, 1001, 10)

avgs = numpy.load(f"data/holstein_semicl/classical_avgs_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy")

panda = pandas.DataFrame(data={"voltage": voltages,
                               "current_L": avgs[:,0,0]*1e3,
                               "u_current_L": avgs[:,1,0]*1e3,
                               "current_R": avgs[:,0,1]*1e3,
                               "u_current_R": avgs[:,1,1]*1e3,
                               "X": avgs[:,0,2],
                               "u_X": avgs[:,1,2],
                               "X^2": avgs[:,0,3],
                               "u_X^2": avgs[:,1,3],
                               "P": avgs[:,0,4],
                               "u_P": avgs[:,1,4],
                               "P^2": avgs[:,0,5],
                               "u_P^2": avgs[:,1,5]})

panda.to_csv(f"data/holstein_semicl/classical_bmme_{file_apx()}.csv")

os.rename(f"data/holstein_semicl/classical_avgs_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy", f"data/holstein_semicl/classical_avgs_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}.npy.migrated")

## Migrate old quantum averages

In [9]:
voltages = numpy.arange(0, 1001, 10)

N = 150
N_diag = 250

#rhos = numpy.load(f"data/holstein_semicl/quantum_ss_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}_N{N}~{N_diag}.npy")
other_data = numpy.load(f"data/holstein_semicl/quantum_avgs_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}_N{N}~{N_diag}.npy")

#rpanda = pandas.DataFrame({"voltage": voltages, "rho_ss": rhos})
apanda = pandas.DataFrame({"voltage": voltages,
                           "current_L": other_data[:,0]*1e3,
                           "current_R": other_data[:,1]*1e3,
                           "X": other_data[:,2],
                           "X^2": other_data[:,3],
                           "P": other_data[:,4],
                           "P^2": other_data[:,5],
                           "occupation": other_data[:,6],
                           "excitation": other_data[:,7]})

#rpanda.to_csv(f"data/holstein_semicl/rho_ss_bmme_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}_N{N}~{N_diag}.csv")
apanda.to_csv(f"data/holstein_semicl/quantum_bmme_{file_apx()}_N{N}~{N_diag}.csv")

#os.rename(f"data/holstein_semicl/quantum_ss_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}_N{N}~{N_diag}.npy", f"data/holstein_semicl/quantum_ss_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}_N{N}~{N_diag}.npy.migrated")
os.rename(f"data/holstein_semicl/quantum_avgs_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}_N{N}~{N_diag}.npy", f"data/holstein_semicl/quantum_avgs_omega{omega:.3f}_lamda{lamda:.3f}_Gamma{Gamma:.3f}_N{N}~{N_diag}.npy.migrated")